TOTO予測
===

* [データ収集](001_データ収集.ipynb)で収集したデータを使う。
* [データ加工](002_データ加工.ipynb)で加工済みとする。

In [1]:
import download
import process_data
import pandas as pd
df = download.get_db_data()
df = process_data.processing_data(df)
#display(df)

# 符号化

* チーム名・スタジアム名を符号化
* 勝ち・負け・引き分けを符号化
* 試合日を月のみ取得

In [2]:
home = pd.concat([df["ホーム"],df["アウェイ"]]).drop_duplicates().reset_index(drop=True).to_dict()
team_dict = {v:k for k, v in home.items()} # 辞書のキー・バリュー交換

In [3]:
tmp = [team_dict[df["ホーム"][i]] for i in range(len(df))]
df["home"] = tmp
tmp = [team_dict[df["アウェイ"][i]] for i in range(len(df))]
df["away"] = tmp
#display(df)

In [4]:
studium = df["スタジアム"].drop_duplicates().reset_index(drop=True).to_dict()
studium_dict = {v:k for k, v in studium.items()}

In [5]:
tmp = [studium_dict[df["スタジアム"][i]] for i in range(len(df))]
df["studium"] = tmp

勝敗はホームチームの勝ち(WIN)、負け(LOSE)、引き分け(DRAW)の三種類に符号化する。  
最終出力を獲得勝ち点で出力することを考え、WIN=3/DRAW=1/LOSE=0としておく。

カップ戦とかでPK戦の結果も含まれているが、PKは引き分けととらえることにする。

In [6]:
import re
import numpy as np
wdl = []
# win-draw-lose VゴールとかPKとかはTOTO予想に関係ないので無視
LOSE = 0
DRAW = 1
WIN = 3
OTHER = np.nan
for result in df["スコア"]:
    tmp = re.split("[-()]",result)
    if len(tmp) < 2: # X-Xという形式でないものはスルー
        wdl.append(OTHER)
        continue
    if int(tmp[0]) > int(tmp[1]):
        wdl.append(WIN)
    elif int(tmp[0]) < int(tmp[1]):
        wdl.append(LOSE)
    else:
        wdl.append(DRAW)
df["result"] = wdl
#display(df)

対戦した時期として、月の情報を入れるものとする。

試合日から情報を検出する。

In [7]:
month = []
for match_day in df["試合日"]:
    tmp = match_day.split("/")[0]
    if tmp.isdigit() == True:
        month.append(int(tmp))
    else:
        month.append(np.nan)
df["month"] = month
#display(month)

In [8]:
df = df.rename(columns={"年度":"year"})
#display(df)

使うデータを1992年~2016年に絞る。  
2017年のデータは予測の正確性検証のために別にしておくため

In [9]:
data = df[df["year"] < 2017]

結果の異常値を弾く。

異常値としてNaN(np.nan)を使ったので、NaNを含む列をドロップする。

In [10]:
data = data.dropna()

# いろんな機械学習を試す

フレームワークを使っていろいろ試してみたけど、うまくいかなかった話。

* [ディープラーニング](004_ディープラーニング.ipynb)  
ネットワークが悪いのかもしれないけど、精度21%程度にしかならなかった。
* [ベイズ分類器](005_ベイズ分類器.ipynb)  
最も精度の良かったベルヌーイ分布でも41%程度…。

# 条件付き確率で考える

単純ベイズ推定器を使ったクラスタリングをベースに、各条件における勝ち・負け・引き分けになる確率を計算し、それぞれを掛け合わせて一番高い確率のものを採用することにする。

## TOTO販売スケジュール取得

まず、確率計算する対象をTOTOの販売スケジュールから取得する

* [TOTO予定データの取得](003_TOTO予定データの取得.ipynb)より取得

In [11]:
toto, miniA, miniB, goal = download.get_toto_schedule(1048)
display(toto)

,開催日,試合開始予定時間,競技場,ホーム,アウェイ
1,10/13,15:00,長良川,岐阜,岡山
2,10/14,14:00,ニッパツ,横浜FM,鹿島
3,10/14,16:00,ＢＭＷス,湘南,柏
4,10/13,15:30,維新公園,山口,福岡
5,10/13,16:00,ＮＡＣＫ,大宮,栃木
6,10/13,15:00,ニンスタ,愛媛,横浜FC
7,10/14,14:00,Ｋｓスタ,水戸,東京Ｖ
8,10/14,14:00,石川西部,金沢,松本
9,10/14,15:00,町田,町田,大分
10,10/13,17:00,西京極,京都,徳島


## 検討する条件

1. チームの通算成績における勝率(ベースの分布？)  
$\Rightarrow \dfrac{勝利した数}{通算試合数}$
2. 対戦相手を限定した場合の勝率  
$\Rightarrow \dfrac{対戦相手に勝利した数}{対戦相手との通算試合数}$
3. 試合開催が何月かで限定した場合の勝率  
$\Rightarrow \dfrac{〇月の試合の中で勝利した数}{〇月に行われた試合数}$
4. 試合開催がどのスタジアムで限定した勝率  
$\Rightarrow \dfrac{指定スタジアムでの勝利数}{指定スタジアムでの試合数}$

In [12]:
def get_team_data(team, df):
    """
    指定したチームに関するデータのみ抽出する
    """
    team_data = df[(df["ホーム"]==team) | (df["アウェイ"]==team)]
    return team_data

def get_win_data(team, df, result=3):
    """
    指定したチームが勝利したデータのみ抽出する
    引数resultを指定すると、引き分け/負けのデータも収集可能
    """
    if result == 3:
        win_data = df[((df["ホーム"]==team) & (df["result"]==3)) | ((df["アウェイ"]==team) & (df["result"]==0))]
    elif result == 1:
        win_data = df[(df["result"]==1)]
    elif result == 0:
        win_data = df[((df["ホーム"]==team) & (df["result"]==0)) | ((df["アウェイ"]==team) & (df["result"]==3))]        
    return win_data

def get_studium_data(studium, df):
    """
    指定したスタジアムでのデータのみ抽出する
    """
    studium_data = df[(df["スタジアム"]==studium)]
    return studium_data

def get_month_data(month, df):
    """
    指定した月のデータのみ抽出する
    """
    month_data = df[df["month"]==month]
    return month_data

In [72]:
def proba_all(team, data, index="all"):
    """
    全試合に関する勝率を計算
    """
    team_data = get_team_data(team, data)
    if len(team_data)==0:
        return None
    win_data = get_win_data(team, team_data, result=3)
    draw_data = get_win_data(team, team_data, result=1)
    lose_data = get_win_data(team, team_data, result=0)
    ret = pd.DataFrame()
    ret["win"] = [len(win_data)/len(team_data)]
    ret["draw"] = [len(draw_data)/len(team_data)]
    ret["lose"] = [len(lose_data)/len(team_data)]
    #print(len(win_data),len(draw_data),len(lose_data),len(team_data))
    ret = ret.rename(index={0:index})
    return ret

def proba_oppotunity(team, oppotunity, data):
    """
    対戦相手を限定した際の勝率を計算
    """
    team_data = get_team_data(team, data)
    team_data = get_team_data(oppotunity, team_data)
    return proba_all(team, team_data,index="oppotunity")

def proba_month(team, month, data):
    """
    開催日が何月かで限定した際の勝率を計算
    """
    team_data = get_team_data(team, data)
    month_data = get_month_data(month, team_data)
    #display(month_data)
    return proba_all(team,month_data,index="month")

def proba_quarter(team, month, data):
    """
    開催月を四半期ごとに分けて勝率を計算
    Jリーグはだいたい2月終わり～12月初めまで。12月からは天皇杯。
    """
    QUARTERS = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
    if month >= 1 or month <= 3:
        q = 0
    elif month >= 4 and month <= 6:
        q = 1
    elif month >= 7 and month <= 9:
        q = 2
    else:
        q = 3
    team_data = get_team_data(team, data)
    quarter_data = data[(data["month"]==QUARTERS[q][0]) | (data["month"]==QUARTERS[q][1]) | (data["month"]==QUARTERS[q][2])]
    #display(quarter_data)
    return proba_all(team,quarter_data,index="quarter")

def proba_studium(team, studium, data):
    """
    指定スタジアムでの試合に限定した勝率を計算
    """
    team_data = get_team_data(team, data)
    studium_data = get_studium_data(studium, team_data)
    return proba_all(team, studium_data,index="studium")

HOME = "川崎Ｆ"
AWAY = "札幌"
STUDIUM = "等々力"
tmp = pd.DataFrame()
tmp = tmp.append(proba_all(HOME, df))
tmp = tmp.append(proba_oppotunity(HOME, AWAY, df))
tmp = tmp.append(proba_month(HOME,9,df))
tmp = tmp.append(proba_studium(HOME, STUDIUM, df))
display(tmp.product().idxmax())

'win'

In [73]:
def prediction(home, away, month, studium, df):
    tmp = pd.DataFrame()
    proba = proba_all(home, df)
    if proba is not None:
        tmp = tmp.append(proba)
    proba = proba_oppotunity(home, away, df)
    if proba is not None:
        tmp = tmp.append(proba)
#    proba = proba_month(home, month, df)
#    if proba is not None:
#        tmp = tmp.append(proba)
    proba = proba_quarter(home, month, df)
    if proba is not None:
        tmp = tmp.append(proba)
    proba = proba_studium(home, studium, df)
    if proba is not None:
        tmp = tmp.append(proba)
    return tmp.product().idxmax(), tmp

In [74]:
result, pred = prediction("川崎Ｆ","札幌",9,"等々力",df)
print(result)
display(pred)

win


,win,draw,lose
all,0.510230,0.186701,0.303069
oppotunity,0.761905,0.190476,0.047619
quarter,0.384615,0.269231,0.346154
studium,0.593176,0.186352,0.220472


## アウェイチームから見た場合のデータも考える

ここまでだとホームチームから見た場合の確率しか考えていないので、アウェイチームから見た場合も考えてみる。  
確率の計算としては、単にHOME/AWAYを入れ替えるだけで同じような計算が可能。  
全く同じ方法で当てはめてみる。

In [75]:
result, pred = prediction("札幌","川崎Ｆ",9,"等々力",df)
print(result)
display(pred)

lose


,win,draw,lose
all,0.374554,0.219976,0.405470
oppotunity,0.047619,0.190476,0.761905
quarter,0.386364,0.193182,0.420455
studium,0.083333,0.083333,0.833333


## 結果の評価を行う

結果はホーム側からみた予想とアウェイ側から見た予想が双方出てくるので、下のような関係性で最終結果を出すものとする。

|アウェイ\ホーム|WIN|DRAW|LOSE|
|-------------|----|----|----|
|WIN          |DRAW|LOSE|LOSE|
|DRAW         |WIN |DRAW|LOSE|
|LOSE         |WIN |WIN |DRAW|

※最終結果はホームチームから見た結果

2016年以前のデータを使い、2017年のデータを予測して、精度を評価する。  
対象はJ1/J2とする。

In [76]:
data = df[df["year"] < 2017]
data = data.dropna()
answer = df[(df["year"] == 2017) & ((df["大会"]=="Ｊ１") | (df["大会"]=="Ｊ２"))].reset_index()
answer["result"] = answer["result"].replace(3,"win")
answer["result"] = answer["result"].replace(1,"draw")
answer["result"] = answer["result"].replace(0,"lose")
#display(answer)

In [77]:
from tqdm import tqdm_notebook
pred_home = []
pred_away = []
pred_result = []
for i in tqdm_notebook(range(len(answer))):
    tmp1, tmp2 = prediction(answer["ホーム"].iloc[i],answer["アウェイ"].iloc[i],answer["month"].iloc[i],answer["スタジアム"].iloc[i],data)
    tmp3, tmp4 = prediction(answer["アウェイ"].iloc[i],answer["ホーム"].iloc[i],answer["month"].iloc[i],answer["スタジアム"].iloc[i],data)
    pred_home.append(tmp1)
    pred_away.append(tmp3)
    if pred_home[i] == pred_away[i]:
        pred_result.append("draw")
    elif pred_home[i] == "win" or (pred_home[i] == "draw" and pred_away[i] == "lose"):
        pred_result.append("win")
    else:
        pred_result.append("lose")
        
answer["予想1"] = pred_home
answer["予想2"] = pred_away
answer["最終予想"] = pred_result
#display(answer)
print(len(answer[answer["result"]==answer["最終予想"]])/len(answer))


0.3815104166666667


結果、38%という数値になった。  
何も考えずに予想すると33%ですが、この場合の期待値はどうなる？

## TOTOの予定表から予測させる

TOTOの予定表から対象となるゲームのホーム・アウェイ・競技場を収集し、predictionに渡すことで結果を予想する。  

In [60]:
toto, miniA, miniB, goal = download.get_toto_schedule(1049)
#display(toto)

In [61]:
pred_home = []
pred_away = []
pred_result = []
for i in range(len(toto)):
    #print(toto["ホーム"].iloc[i],toto["アウェイ"].iloc[i],int(toto["開催日"].iloc[i].split("/")[0]),toto["競技場"].iloc[i],end="")
    tmp1, tmp2 = prediction(toto["ホーム"].iloc[i],toto["アウェイ"].iloc[i],int(toto["開催日"].iloc[i].split("/")[0]),toto["競技場"].iloc[i],df)
    tmp3, tmp4 = prediction(toto["アウェイ"].iloc[i],toto["ホーム"].iloc[i],int(toto["開催日"].iloc[i].split("/")[0]),toto["競技場"].iloc[i],df)
    #print(tmp)
    pred_home.append(tmp1)
    pred_away.append(tmp3)
    if pred_home[i] == pred_away[i]:
        pred_result.append("draw")
    elif pred_home[i] == "win" or (pred_home[i] == "draw" and pred_away[i] == "lose"):
        pred_result.append("win")
    else:
        pred_result.append("lose")
        
toto["予想1"] = pred_home
toto["予想2"] = pred_away
toto["最終予想"] = pred_result
toto

,開催日,試合開始予定時間,競技場,ホーム,アウェイ,7,予想1,予想2,最終予想
1,10/20,14:00,ユアスタ,仙台,鳥栖,データ,win,lose,win
2,10/20,14:00,アイスタ,清水,広島,データ,win,lose,win
3,10/20,15:00,吹田Ｓ,Ｇ大阪,横浜FM,データ,lose,win,lose
4,10/20,16:00,埼玉,浦和,鹿島,データ,win,win,draw
5,10/20,16:00,ＢＭＷス,湘南,札幌,データ,lose,win,lose
6,10/20,19:00,味スタ,FC東京,Ｃ大阪,データ,win,win,draw
7,10/20,19:00,等々力,川崎Ｆ,神戸,データ,win,lose,win
8,10/21,16:00,トラスタ,長崎,磐田,データ,lose,win,lose
9,10/21,16:00,Ｃスタ,岡山,町田,データ,draw,draw,draw
10,10/20,15:00,西京極,京都,新潟,データ,win,lose,win


## TOTOの結果を読み取る

既に結果が出ているTOTOは結果を取得するようにする。  
予想が正しいかを過去のデータから検算し、この予想方式の確度を計算する。

※今のところ途中  
W杯の予想とかが入ってきて面倒だったため

In [19]:
def get_toto_result(holdCntId):
    url = "http://sport-kuji.toto-dream.com/dci/I/IPB/IPB01.do?op=lnkHoldCntLotResultLsttoto&holdCntId=" + str(holdCntId)
    dfs = pd.io.html.read_html(url)
    point = dfs[2][1:][6].reset_index(drop=True)[1:]
    point[point==1]="win"
    point[point==0]="draw"
    point[point==2]="lose"
    result = dfs[2][1:][4].reset_index(drop=True)[1:]
    return point, result

In [20]:
point, result = get_toto_result(1041)
toto["結果"] = result
toto["result"] = point
len(toto[toto["予想1"]==toto["result"]])/len(toto)

0.3076923076923077

In [21]:
result = []
for i in range(1025,1047):
    toto, miniA, miniB, goal = download.get_toto_schedule(i)
    if toto is not None:
        predict1 = []
        predict2 = []
        for j in range(len(toto)):
            #print(toto["ホーム"].iloc[i],toto["アウェイ"].iloc[i],9,toto["競技場"].iloc[i],end="")
            tmp1, tmp2 = prediction(toto["ホーム"].iloc[j],toto["アウェイ"].iloc[j],9,toto["競技場"].iloc[j],df)
            tmp3, tmp4 = prediction(toto["アウェイ"].iloc[j],toto["ホーム"].iloc[j],9,toto["競技場"].iloc[j],df)
            #print(tmp)
            predict1.append(tmp1)
            predict2.append(tmp3)
        toto["予想1"] = predict1
        toto["予想2"] = predict2
        point, result = get_toto_result(i)
        toto["結果"] = result
        toto["result"] = point
        print([i,len(toto[toto["予想1"]==toto["result"]])])

[1025, 3]
[1027, 4]
[1029, 6]
[1031, 0]
[1033, 10]
[1034, 4]
[1036, 4]
[1038, 6]
[1039, 7]
[1041, 7]
[1042, 3]
[1043, 6]
[1045, 6]
[1046, 7]


# Football-Labのデータから予測

攻撃と守備は表裏一体なので、攻撃に関してはチャンス構築率、守備に関しては被チャンス構築率を見ることにし、最も値の良いチームのデータを1とした形に正規化する。  
それぞれの正規化データを掛け合わせて、チームとしての攻撃/守備がどの程度かを見てみる。  
なお、最下位チームの正規化データは0になってしまい掛け算できないので、0.1ずつ下駄をはかせてみている。

In [22]:
fldata = download.fl_get_all_data()
fldata = fldata.iloc[:,np.r_[0:2,12:22]]
#display(fldata)

In [23]:
fldata["attack"] = [float(fldata["チャンス構築率"][i].replace("%","")) for i in range(len(fldata))]
fldata["defense"] = [float(fldata["被チャンス構築率"][i].replace("%","")) for i in range(len(fldata))]

In [24]:
import scipy.stats as stats
fldata["attack"]= [(fldata["attack"][i] - min(fldata["attack"])+0.1)/(max(fldata["attack"])+0.1 - min(fldata["attack"])+0.1) for i in range(len(fldata))]
fldata["defense"]= [1.0 - (fldata["defense"][i] - min(fldata["defense"])+0.1)/(max(fldata["defense"])+0.1 - min(fldata["defense"])+0.1) for i in range(len(fldata))]

In [25]:
fldata["point"] = fldata["attack"] * fldata["defense"]

In [26]:
fldata.sort_values("point",ascending=False)

,順位,チーム,攻撃回数,シュート,チャンス構築率,ゴール,シュート成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被シュート成功率,attack,defense,point
0,1,川崎Ｆ,115.0,15.7,13.6%,1.4,9.3%,114.7,9.0,7.8%,0.7,8.0%,0.977778,0.983871,0.962007
5,6,浦和,115.0,13.9,12.1%,1.4,10.0%,115.8,10.9,9.4%,1.0,9.5%,0.644444,0.725806,0.467742
7,8,Ｃ大阪,121.5,14.9,12.3%,1.2,7.9%,120.7,12.4,10.2%,1.1,8.7%,0.688889,0.596774,0.411111
2,3,鹿島,124.8,13.6,10.9%,1.4,10.6%,122.4,10.7,8.7%,1.1,10.0%,0.377778,0.838710,0.316846
14,15,磐田,120.4,13.7,11.4%,1.1,7.8%,117.9,11.6,9.9%,1.5,12.9%,0.488889,0.645161,0.315412
3,4,FC東京,118.4,13.3,11.2%,1.2,8.8%,121.4,11.7,9.6%,0.9,7.7%,0.444444,0.693548,0.308244
1,2,広島,118.5,13.8,11.6%,1.4,10.5%,118.2,12.3,10.4%,0.8,6.7%,0.533333,0.564516,0.301075
9,10,横浜FM,124.7,14.8,11.8%,1.8,12.1%,120.8,13.3,11.1%,1.6,11.9%,0.577778,0.451613,0.260932
13,14,柏,127.0,14.5,11.4%,1.3,8.8%,126.6,13.5,10.7%,1.5,11.2%,0.488889,0.516129,0.252330
10,11,神戸,115.4,13.1,11.4%,1.1,8.7%,115.2,13.2,11.5%,1.4,10.4%,0.488889,0.387097,0.189247


結果、実際の順位表との乖離は多少なりあるものの、戦力的にはそこそこそれらしいのでは？という感じがする結果になった。  
15位の磐田が4番目なのは、チームとしては戦えているが決定力が低いためと思われる。逆に最下位の名古屋は決定力が高くて救われているが、あまりチームとしては戦えていないということなのかもしれない。  
なんだかんだでシュートが入るかどうか(FWの決定力に依存する？)も勝敗に非常に効くので、攻撃のほうにシュート成功率も入れた方が良いかも。

うまいこと現状のチーム状況を比較しやすい指標ができれば、そこから勝率を計算することも考えてみたいが、それだとシーズンが進まないと予想できないので、あまり意味がないかも？